In [21]:
import pandas as pd
from pathlib import Path

# Set correct INPUT_FILE and OUTPUT_FILE (discovered in workspace)
INPUT_FILE = Path(r"C:\Users\peace\Downloads\Datasets\Retail dataset\online_retail_II.xlsx")
OUTPUT_FILE = Path(r"C:\Users\peace\Downloads\Datasets\Retail dataset\data\raw\online_retail_dirty.csv")
OUTPUT_FILE.parent.mkdir(parents=True, exist_ok=True)

# Load the excel file (also keep df variable)
df = pd.read_excel(INPUT_FILE, engine='openpyxl')

# Quick sanity check
print(f"INPUT_FILE: {INPUT_FILE}")
print(df.head())
print(df.info())

# --- Quick sample from the dirty output (if it exists) ---
try:
    dirty_path = OUTPUT_FILE
    if dirty_path.exists():
        df_dirty = pd.read_csv(dirty_path, parse_dates=['InvoiceDate'], encoding='ISO-8859-1', low_memory=False)
        print('\nLoaded dirty dataset sample:')
        print(df_dirty.head().to_string())

        # summary counts for injected issues
        cust_col = next((c for c in ['CustomerID', 'Customer ID'] if c in df_dirty.columns), None)
        price_col = next((c for c in ['UnitPrice', 'Price'] if c in df_dirty.columns), None)
        print(f"\nRows: {len(df_dirty)}")
        print('Missing CustomerID:', df_dirty[cust_col].isna().sum() if cust_col else 'N/A')
        print('Missing InvoiceDate:', df_dirty['InvoiceDate'].isna().sum() if 'InvoiceDate' in df_dirty.columns else 'N/A')
        print('Negative Quantity:', (df_dirty['Quantity'] < 0).sum() if 'Quantity' in df_dirty.columns else 'N/A')
        print('Negative Price:', (df_dirty[price_col] < 0).sum() if price_col else 'N/A')
        print('Duplicate rows:', df_dirty.duplicated().sum())
    else:
        print(f"Dirty output not found at: {dirty_path}")
except Exception as e:
    print('Error loading dirty sample:', e)


INPUT_FILE: C:\Users\peace\Downloads\Datasets\Retail dataset\online_retail_II.xlsx
  Invoice StockCode                          Description  Quantity  \
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1  489434    79323P                   PINK CHERRY LIGHTS        12   
2  489434    79323W                  WHITE CHERRY LIGHTS        12   
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   

          InvoiceDate  Price  Customer ID         Country  
0 2009-12-01 07:45:00   6.95      13085.0  United Kingdom  
1 2009-12-01 07:45:00   6.75      13085.0  United Kingdom  
2 2009-12-01 07:45:00   6.75      13085.0  United Kingdom  
3 2009-12-01 07:45:00   2.10      13085.0  United Kingdom  
4 2009-12-01 07:45:00   1.25      13085.0  United Kingdom  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null

In [20]:
"""
Purpose:
--------
Introduce controlled, realistic noise into the UCI Online Retail dataset
to simulate real-world data quality issues commonly found in e-commerce
analytics pipelines.

This intentionally dirty dataset will later be cleaned using SQL
(PostgreSQL) and used for Marketing Mix Modeling (MMM).

Author: Peace Odum
"""

import pandas as pd
import numpy as np
from pathlib import Path


# -------------------------------
# Configuration
# -------------------------------

RANDOM_SEED = 42

# Debug: show existing INPUT_FILE if present
print("DEBUG - INPUT_FILE in globals:", 'INPUT_FILE' in globals())
if 'INPUT_FILE' in globals():
    try:
        print("DEBUG - INPUT_FILE:", INPUT_FILE)
        print("DEBUG - exists:", INPUT_FILE.exists())
        print("DEBUG - suffix:", INPUT_FILE.suffix)
    except Exception as e:
        print("DEBUG - could not inspect INPUT_FILE:", e)

# Use existing INPUT_FILE if already set by the auto-detect cell; otherwise fall back
if 'INPUT_FILE' not in globals():
    try:
        INPUT_FILE = PROJECT_ROOT / "data" / "raw" / "online_retail.csv"
        OUTPUT_FILE = PROJECT_ROOT / "data" / "raw" / "online_retail_dirty.csv"
    except NameError:
        INPUT_FILE = Path("data/raw/online_retail.csv")
        OUTPUT_FILE = Path("data/raw/online_retail_dirty.csv")

GERMANY_ALIASES = ["GER", "DE", "Deutschland", "Germany "]


# -------------------------------
# Utility Functions
# -------------------------------

def set_random_seed(seed: int = 42) -> None:
    """Ensure reproducibility."""
    np.random.seed(seed)


def load_data(path: Path) -> pd.DataFrame:
    """Load raw online retail data."""
    # Auto-detect CSV vs Excel by suffix
    path = Path(path)
    if path.suffix.lower() in ['.xls', '.xlsx']:
        df = pd.read_excel(path, engine='openpyxl')
    else:
        df = pd.read_csv(path, encoding='ISO-8859-1')
    df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"], errors="coerce")
    return df


# -------------------------------
# Noise Injection Functions
# -------------------------------

def introduce_missing_customer_ids(df: pd.DataFrame, pct: float = 0.08) -> None:
    mask = np.random.rand(len(df)) < pct
    if 'CustomerID' in df.columns:
        df.loc[mask, "CustomerID"] = np.nan
    elif 'Customer ID' in df.columns:
        df.loc[mask, "Customer ID"] = np.nan


def introduce_missing_invoice_dates(df: pd.DataFrame, pct: float = 0.03) -> None:
    mask = np.random.rand(len(df)) < pct
    if 'InvoiceDate' in df.columns:
        df.loc[mask, "InvoiceDate"] = pd.NaT


def introduce_returns(df: pd.DataFrame, pct: float = 0.05) -> None:
    mask = np.random.rand(len(df)) < pct
    if 'Quantity' in df.columns:
        df.loc[mask, "Quantity"] = df.loc[mask, "Quantity"] * -1


def introduce_price_errors(df: pd.DataFrame, neg_pct: float = 0.02, zero_pct: float = 0.02) -> None:
    neg_mask = np.random.rand(len(df)) < neg_pct
    zero_mask = np.random.rand(len(df)) < zero_pct

    if 'UnitPrice' in df.columns:
        df.loc[neg_mask, "UnitPrice"] = df.loc[neg_mask, "UnitPrice"] * -1
        df.loc[zero_mask, "UnitPrice"] = 0
    elif 'Price' in df.columns:
        df.loc[neg_mask, "Price"] = df.loc[neg_mask, "Price"] * -1
        df.loc[zero_mask, "Price"] = 0


def corrupt_invoice_numbers(df: pd.DataFrame, pct: float = 0.02) -> None:
    mask = np.random.rand(len(df)) < pct
    suffixes = np.random.choice(["X", "ERR", "?"], size=mask.sum())
    if 'InvoiceNo' in df.columns:
        df.loc[mask, "InvoiceNo"] = (
            df.loc[mask, "InvoiceNo"].astype(str) + suffixes
        )
    elif 'Invoice' in df.columns:
        df.loc[mask, "Invoice"] = (
            df.loc[mask, "Invoice"].astype(str) + suffixes
        )


def duplicate_rows(df: pd.DataFrame, pct: float = 0.03) -> pd.DataFrame:
    dup_rows = df.sample(frac=pct, random_state=RANDOM_SEED)
    return pd.concat([df, dup_rows], ignore_index=True)

# Country inconsistencies are intentionally introduced ONLY for Germany,
# as Germany is the target market for downstream MMM analysis.

def introduce_country_inconsistencies(df: pd.DataFrame, pct: float = 0.10) -> None:
    if 'Country' not in df.columns:
        return
    germany_mask = df["Country"] == "Germany"
    noise_mask = germany_mask & (np.random.rand(len(df)) < pct)

    df.loc[noise_mask, "Country"] = np.random.choice(
        GERMANY_ALIASES,
        size=noise_mask.sum()
    )


def introduce_outliers(df: pd.DataFrame, pct: float = 0.01) -> None:
    mask = np.random.rand(len(df)) < pct
    if 'Quantity' in df.columns:
        df.loc[mask, "Quantity"] = df.loc[mask, "Quantity"] * 20


# -------------------------------
# Main Pipeline
# -------------------------------

def main() -> None:
    global OUTPUT_FILE
    set_random_seed(RANDOM_SEED)

    # Use the discovered Excel file directly to avoid stray overrides
    df = load_data(Path(r"C:\Users\peace\Downloads\Datasets\Retail dataset\online_retail_II.xlsx"))

    # Standardize column names used by noise functions
    df.columns = [c.strip() for c in df.columns]
    rename_map = {}
    if 'Invoice' in df.columns:
        rename_map['Invoice'] = 'InvoiceNo'
    if 'Price' in df.columns:
        rename_map['Price'] = 'UnitPrice'
    if 'Customer ID' in df.columns:
        rename_map['Customer ID'] = 'CustomerID'
    if rename_map:
        df = df.rename(columns=rename_map)

    introduce_missing_customer_ids(df)
    introduce_missing_invoice_dates(df)
    introduce_returns(df)
    introduce_price_errors(df)
    corrupt_invoice_numbers(df)
    introduce_country_inconsistencies(df)

    df = duplicate_rows(df)
    introduce_outliers(df)

    # Shuffle rows to break natural ordering
    df = df.sample(frac=1, random_state=RANDOM_SEED).reset_index(drop=True)

    # Ensure OUTPUT_FILE is set to a sensible path
    if 'OUTPUT_FILE' not in globals():
        OUTPUT_FILE = Path(r"C:\Users\peace\Downloads\Datasets\Retail dataset\data\raw\online_retail_dirty.csv")

    OUTPUT_FILE.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(OUTPUT_FILE, index=False)

    print(f"Dirty dataset successfully written to: {OUTPUT_FILE}")


if __name__ == "__main__":
    main()


DEBUG - INPUT_FILE in globals: True
DEBUG - INPUT_FILE: C:\Users\peace\Downloads\Datasets\Retail dataset\data\raw\online_retail.csv
DEBUG - exists: False
DEBUG - suffix: .csv
Dirty dataset successfully written to: C:\Users\peace\Downloads\Datasets\Retail dataset\data\raw\online_retail_dirty.csv


In [22]:
import pandas as pd
from pathlib import Path

# Load dirty dataset
dirty_path = OUTPUT_FILE if 'OUTPUT_FILE' in globals() else Path(r"C:\Users\peace\Downloads\Datasets\Retail dataset\data\raw\online_retail_dirty.csv")
print('Dirty path:', dirty_path)
df_dirty = pd.read_csv(dirty_path, parse_dates=['InvoiceDate'], encoding='ISO-8859-1', low_memory=False)

df_dirty['Country_clean'] = df_dirty['Country'].astype(str).str.strip()

print('\nTop country values:')
print(df_dirty['Country_clean'].value_counts().head(40).to_string())

aliases = ['ger', 'de', 'deutschland', 'germany']
mask = df_dirty['Country_clean'].str.lower().isin(aliases)

print(f"\nRows matching Germany aliases (case-insensitive): {mask.sum()}")
print('Unique matched values:', sorted(df_dirty.loc[mask, 'Country_clean'].unique().tolist()))

print('\nSample rows with Germany variants:')
key_invoice = 'InvoiceNo' if 'InvoiceNo' in df_dirty.columns else ('Invoice' if 'Invoice' in df_dirty.columns else None)
cols = [c for c in [key_invoice, 'Country_clean'] if c]
print(df_dirty.loc[mask, cols].head(20).to_string())


Dirty path: C:\Users\peace\Downloads\Datasets\Retail dataset\data\raw\online_retail_dirty.csv

Top country values:
Country_clean
United Kingdom          500407
EIRE                      9960
Germany                   7736
France                    5941
Netherlands               2862
Spain                     1311
Switzerland               1223
Portugal                  1140
Belgium                   1082
Channel Islands            936
Sweden                     929
Italy                      754
Australia                  678
Cyprus                     569
Austria                    558
Greece                     535
Denmark                    446
United Arab Emirates       438
Norway                     375
Finland                    367
Unspecified                317
USA                        253
Deutschland                232
Japan                      230
GER                        221
Poland                     203
DE                         192
Malta                      175
Lit